In [26]:
import json

basin = 'stanislaus'

with open('../{}/pywr_model.json'.format(basin), 'r') as f:
    m = json.load(f)

new_names = {}
for n in m['nodes']:
    node_name = n['name']
    if node_name[-7:] == ' [link]':
        n['name'] = node_name.replace(' [link]', '')
        node_type = 'link'
    else:
        node_type = 'node'
    metadata = json.loads(n.get('comment', '{}'))
    if 'keep' in metadata:
        metadata['keep'] = True
    metadata['resource_class'] = node_type
    n['comment'] = json.dumps(metadata)
#         n['name'] = 'link/' + node_name.replace(' [link]', '')
#     else:
#         n['name'] = 'node/' + node_name
    new_names[node_name] = n['name']
    
    for k in n:
        v = n[k]
        if type(v) == str and v[:5] in ['node/', 'link/']:
            n[k] = v[5:]
        elif type(v) == list:
            for j, l in enumerate(v):
                if type(l) == str and l[:5] in ['node/', 'link/']:
                    n[k][j] = l[5:]


for e in m['edges']:
    e[0] = new_names[e[0]]
    e[1] = new_names[e[1]]

for section in ['parameters', 'recorders']:
    new_dict = {}
    for key, val in m[section].items():
        for k in val:
            v = val[k]
            if k == 'node':
                val['node'] = new_names[v]
            elif type(v) == str and v[:5] in ['node/', 'link/']:
                val[k] = v[5:]
            elif type(v) == list:
                for j, l in enumerate(v):
                    if type(l) == str and l[:5] in ['node/', 'link/']:
                        val[k][j] = l[5:]
        
        if key[:4] in ['node', 'link']:
            key = key[5:]
        
        new_dict[key] = val
    m[section] = new_dict

with open('../{}/pywr_model_cleaned.json'.format(basin), 'w') as f:
    json.dump(m, f, indent=4)